In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3

In [6]:
datagen = ImageDataGenerator(
                rescale=1./255,
                horizontal_flip=True,
                rotation_range=0.2)

In [7]:
train_generator = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 1506 images belonging to 3 classes.


In [8]:
val_generator = train_datagen.flow_from_directory(
        'dataset/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 215 images belonging to 3 classes.


In [9]:
test_generator = train_datagen.flow_from_directory(
        'dataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 431 images belonging to 3 classes.


In [10]:
class_names = list(train_generator.class_indices.keys())
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

## Building the Model

In [11]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax'),
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        3

In [13]:
# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [14]:
len(train_generator)

48

In [15]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    batch_size=32,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1,
    epochs=20,
)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


Epoch 1/20
48/48 [==============================] - 158s 3s/step - loss: 0.9073 - accuracy: 0.4827 - val_loss: 0.8836 - val_accuracy: 0.4651
Epoch 2/20
48/48 [==============================] - 157s 3s/step - loss: 0.6798 - accuracy: 0.6892 - val_loss: 0.5955 - val_accuracy: 0.7721
Epoch 3/20
48/48 [==============================] - 185s 4s/step - loss: 0.4105 - accuracy: 0.8293 - val_loss: 0.6096 - val_accuracy: 0.7488
Epoch 4/20
48/48 [==============================] - 161s 3s/step - loss: 0.3403 - accuracy: 0.8659 - val_loss: 0.3617 - val_accuracy: 0.8465
Epoch 5/20
48/48 [==============================] - 166s 3s/step - loss: 0.3275 - accuracy: 0.8645 - val_loss: 0.3374 - val_accuracy: 0.8465
Epoch 6/20
48/48 [==============================] - 166s 3s/step - loss: 0.2785 - accuracy: 0.8951 - val_loss: 0.2714 - val_accuracy: 0.8977
Epoch 7/20
48/48 [==============================] - 168s 3s/step - loss: 0.2617 - accuracy: 0.9004 - val_loss: 0.4437 - val_accuracy: 0.8047
Epoch 8/20
48

In [16]:
scores = model.evaluate(test_generator)
scores

14/14 [==============================] - 32s 2s/step - loss: 0.1549 - accuracy: 0.9582


[0.15492446720600128, 0.9582366347312927]